# LangChain Middleware

Ref: https://docs.langchain.com/oss/python/langchain/middleware/overview

Middleware allows you to control and customize agent execution at every step.


## Setup

Configure `.env` before running. See `.env.sample`.


In [ ]:
import rich
from dotenv import load_dotenv

load_dotenv()

## Custom Middleware: Logging All Hooks

Create a middleware that logs every hook to understand the execution flow.


In [ ]:
from collections.abc import Callable
from datetime import datetime
from typing import Any

from langchain.agents import create_agent
from langchain.agents.middleware.types import (
    AgentMiddleware,
    AgentState,
    ToolCallRequest,
)
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langgraph.graph.state import CompiledStateGraph
from langgraph.runtime import Runtime
from langgraph.types import Command


class LoggingMiddleware(AgentMiddleware[AgentState[Any], None]):
    """Middleware that logs all hooks."""

    def before_agent(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        rich.print("[bold blue]>>> before_agent[/bold blue]")
        rich.print("state =", state)
        rich.print("runtime =", runtime)
        return None

    def before_model(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        rich.print("[bold green]>>> before_model[/bold green]")
        rich.print("state =", state)
        rich.print("runtime =", runtime)
        return None

    def after_model(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        rich.print("[bold green]<<< after_model[/bold green]")
        rich.print("state =", state)
        rich.print("runtime =", runtime)
        return None

    def wrap_tool_call(
        self,
        request: ToolCallRequest,
        handler: Callable[[ToolCallRequest], ToolMessage | Command[Any]],
    ) -> ToolMessage | Command[Any]:
        rich.print("[bold yellow]>>> wrap_tool_call (before)[/bold yellow]")
        rich.print("request =", request)
        result = handler(request)
        rich.print("[bold yellow]<<< wrap_tool_call (after)[/bold yellow]")
        rich.print("result =", result)
        return result

    def after_agent(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        rich.print("[bold blue]<<< after_agent[/bold blue]")
        rich.print("state =", state)
        rich.print("runtime =", runtime)
        return None


@tool
def get_current_time() -> str:
    """Get the current time."""
    return datetime.now().isoformat()


model = ChatAnthropic(model="claude-sonnet-4-5-20250929")
agent: CompiledStateGraph[Any] = create_agent(
    model=model,
    tools=[get_current_time],
    system_prompt="You are a helpful assistant.",
    middleware=[LoggingMiddleware()],
)

In [ ]:
response = agent.invoke({"messages": [{"role": "user", "content": "What time is it?"}]})
rich.print("response =", response)